# Forest Inventory with Airborne LiDAR

Through this practical, you will use metrics derived from LiDAR point clouds to estimate stem volume (m3 ha-1), basal area (m2 ha-1), and diameter at breast height (DBH; cm). You have been provided with 166 forest plots dominated by Sitka spruce plantations.

## LiDAR Data Analysis

Given the volume of LiDAR required to cover the full area of this forest, it would be too much for you to process as part of this tutorial. Therefore, the LiDAR point cloud analysis has already been applied but consists of the following steps, which were applied using the SPDLib software:

 1. Import the LAS/LAZ files into the SPD file format.
 2. Filter noise from the point cloud
 3. Classify ground returns
 4. Populate point height field using the classified ground returns.
 5. Calculate the LiDAR metrics from the point cloud


## Regression Analysis

### Defining the Problem

Airborne LiDAR is a popular tool for forest inventory applications as it provides direct, high resolution measurements of canopy height and cover. However, airborne LiDAR cannot directly measure several important forest biometrics such as stem diameter, basal area, wood volume and biomass. To indirectly derive these forest biometrics from LiDAR measurements requires regression analysis using an area-based approach (White et al., 2013) or a single-tree approach (Breidenbach et al., 2010; Yu et al., 2011). In the area-based approach, LiDAR measurements are spatially gridded and converted into a range of  metrics (height percentiles and density statistics) that quantify the vertical and horizontal distribution of laser returns within the forest canopy. The LiDAR metrics are then regressed against forest inventory measurements recorded at ground level in fixed-area plots. The final outcome of this procedure is a wall-to-wall map of estimated forest inventory attributes across the region surveyed by the airborne LiDAR.

You have been provided with three files:

 1. **Forest_Plot_Metrics.csv** – The field plots with associated LiDAR forest metrics.
 2. **Forest_ALS_Metrics.kea** – An image 20x20 m pixels with the LiDAR metrics for the whole forest area to which the regression relationships developed on the field plots will be applied providing stem volume (m3 ha-1), basal area (m2 ha-1) and DBH (cm) for the whole forest area. Note, this image has been masked to just the forest area.
 3. **Forest_ALS_Valid.kea** - A binary image for the metrics image specifying which pixels should be used for processing (i.e., forest pixels). 

Our research challenge is to generate accurate predictions for all three of these response variables by using the LiDAR metrics as predictor variables. This challenge could be solved by predicting the values individually for each response variable through multiple regression (multiple predictors, one response variable). However, such an approach would not preserve the covariance/correlations between the response variables, therefore we will also use multivariate regression to predict all three response variables simultaneously.

### Assumptions underlying regression analysis

Regardless of whether we are using parametric or non-parametric regression methods, there are several assumptions inherent to regression analysis:
 1. There exists a non-random relationship between out predictor and response variable(s). For example, Ordinary Least Squares (OLS) linear regression assumes a constant monotonic relationship between our predictor(s) and response(s).
 2. Multicollinearity – the regression model should only contain predictor variables that are independent or exogeneous (i.e., they are not a function of predictor variables already present within the model).
 3. Homoskedasticity – the residuals/errors derived from a regression model should have constant variance.
 4. Normality – the residuals/errors derived from a regression model should be normally distributed (Gaussian). With larger training datasets, this assumption becomes less problematic as the residuals will naturally approximate a Gaussian distribution due to the Central Limit Theorem.


# 1. Imports

In [1]:
import os

import pandas

%matplotlib inline
import matplotlib.pyplot as plt

import rsgislib.tools.stats


# 2. Read the input plot data 

In [2]:
# Open the CSV file as a Pandas data frame - the df variable.
df = pandas.read_csv('../data/lidar/Forest_Plot_Metrics.csv')

# 3. Get Variables

In [3]:
# Get a list of the columns within the df dataframe
cols = list(df.columns)

# Get the dependent response column names
dep_vars = cols[3:6]
print("Dependent Variables: ", dep_vars)

# Get the indepedent predictor column names
ind_vars = cols[6:]
print("Independent Variables: ", ind_vars)

Dependent Variables:  ['Mean DBH', 'BA / ha', 'Vol / ha']
Independent Variables:  ['N_Pulses', 'N_Returns', 'First_Return', 'Multi_Return', 'LPI', 'CDensity', 'FCI', 'LCI', 'GapFrac', 'VCC', 'hMean', 'qhMean', 'h25', 'h50', 'h60', 'h70', 'h75', 'h80', 'h90', 'h95', 'h99', 'hmax', 'IQR', 'Skew', 'Kurtosis', 'VDR', 'CanopyRR', 'L_mean', 'L_scale', 'L_skewness', 'L_kurtosis', 'L_variation', 'Closed_Vol', 'Open_Vol', 'Oligophotic_Vol', 'Euphotic_Vol', 'cvm_filled_vol', 'cvm_filled_prop', 'Closed_Prop', 'Open_Prop', 'Oligophotic_Prop', 'Euphotic_Prop', 'p_mean', 'p_scale', 'p_skewness', 'p_kurtosis', 'p_variation', 'chm_rumple', 'chm_ruggedness', 'chm_roughness', 'chm_vf', 'chm_vl', 'chm_vd', 'wv_peaks', 'wv_auc', 'wv_mid', 'wv_min', 'wv_max', 'wv_width', 'wv_prominence', 'wv_midmin', 'wv_midmax', 'wv_minmax', 'h25f', 'h50f', 'h60f', 'h70f', 'h75f', 'h80f', 'h85f', 'h90f', 'h95f', 'h99f', 'L_mean_f', 'L_scale_f', 'L_skewness_f', 'L_kurtosis_f', 'L_variation_f']


# 4. Plotting Variables

A common first step in analysing our data is to visualise it, this can be really help in finding any problems our data and also getting an initial idea of any relationships which might exist within the data. 

To read the CSV file with the plot data we will use the pandas Python module (https://pandas.pydata.org), which is a module specifically designed for tabular data analysis and manipulation (i.e., the kind of analysis you might do in an Excel spreadsheet). Pandas can make tasks such as creating plots relatively easy. 

For our data, where there are 78 independent variables and 3 dependent variables, there are 234 plots to create. Doing this manually would be very time consuming so we will use loops to create those plots.

## 4.1 Create Output Directory


In [4]:
# Define the output directory for the plots
out_plots_dir = 'out_plots'

# Check that the output directory exists and create if not
if not os.path.exists(out_plots_dir):
    os.mkdir(out_plots_dir)

## 4.2 Iterative Through Variables and Create plots

In [5]:
# Turn off interactive mode so plots don't get outputted to the notebook 
# just outputted to the output directory
plt.ioff()

# Loop through the dependent variables
for dep_var in dep_vars:
    # For the output file name create a more useable 
    # string to be used when creating the output file name.
    out_file_name_base = ''
    if dep_var == 'Mean DBH':
        out_file_name_base = 'dbh'
    elif dep_var == 'BA / ha':
        out_file_name_base = 'ba'
    elif dep_var == 'Vol / ha':
        out_file_name_base = 'vol'
    
    # Loop through the independent variables
    for ind_var in ind_vars:
        # Create a scatter plot the two variables.
        df.plot.scatter(x=ind_var, y=dep_var)
        # Create the output file name and path for the plot
        out_plot_file = os.path.join(out_plots_dir, "{}_v_{}.png".format(out_file_name_base, ind_var.lower()))
        # Save the plot to disk.
        plt.savefig(out_plot_file)

/Users/pete/miniforge3/envs/osgeo-env-v3/lib/python3.9/site-packages/pandas/plotting/_matplotlib/core.py:386: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


Have a look through those plots, which variables look like they might be useful for our application? For example, comparing the 95th percentiles for each of the dependent variables:

![Basal Area v 95th Height Percentile](figures/ba_v_h95.png)
![DbH v 95th Height Percentile](figures/dbh_v_h95.png)
![Volume v 95th Height Percentile](figures/vol_v_h95.png)


# 5. Pearson Correlation

The Pearson correlation coefficient measures the strength of a linear relationship between two variables, assuming the variables to be normally distributed (Gaussian). We can calculate the Pearson correlation between each of our predictor and response variables by running the following code. 

*Note. The outputted correlations are saved as a CSV file, which can then be opened in which ever spreadsheet application (e.g., Excel) of your choice.*


In [6]:
# Create a dict for sorting the correlations for each dependent variable
out_data = dict()
# Loop through the dependent variables
for dep_var in dep_vars:
    # Create a list for sorting the output correlation values
    corr_vals = list()
    # Loop through the independent variables
    for ind_var in ind_vars:
        # add the correlation values to the list
        corr_vals.append(df[dep_var].corr(df[ind_var], method='pearson'))
    # Add the list of correlation values to the dict.
    out_data[dep_var] = corr_vals

# Use the dict to create a new pandas dataframe with the index
# (i.e., row names) defined with the independent variables.
df_corr = pandas.DataFrame(data=out_data, index=ind_vars)
# Save the dataframe to a CSV file.
df_corr.to_csv('Forest_Plot_Correlation_Metrics.csv')

In [7]:
# A subset of the data can be viewed in the notebook:
df_corr

,Mean DBH,BA / ha,Vol / ha
N_Pulses,0.049515,0.063979,0.043206
N_Returns,0.179326,0.221542,0.199755
First_Return,-0.578153,-0.735066,-0.686424
Multi_Return,0.578153,0.735066,0.686424
LPI,-0.303634,-0.602969,-0.525768
...,...,...,...
L_mean_f,0.893750,0.847581,0.918863
L_scale_f,0.755496,0.618877,0.596091
L_skewness_f,-0.677335,-0.806945,-0.721413
L_kurtosis_f,0.396643,0.502776,0.519443


## 5.1 Selecting Variables using Correlation

We can see that several LiDAR predictors are weakly correlated (Pearson r < 0.7) with our response variables, therefore we must filter or sort the values to identify the most informative predictors. You can either do that within your spreadsheet application or you can use pandas to do this for you. The following code sort the correlation values for each of the dependent variables, printing the top and bottom 10 variables for each dependent variable:

In [8]:
# Sort by the volume correlation column
df_corr.sort_values('Vol / ha', ascending=False, inplace=True)

# Print the 10 highest values
print("Top 10 with positive correlation (Vol / ha)")
print(df_corr.head(10))
print("\n")

# Sort by the volume correlation column
df_corr.sort_values('Vol / ha', ascending=True, inplace=True)

# Print the 10 lowest values
print("Top 10 with negative correlation (Vol / ha)")
print(df_corr.head(10))
print("\n")

Top 10 with positive correlation (Vol / ha)
          Mean DBH   BA / ha  Vol / ha
chm_vf    0.840862  0.889441  0.944067
h25f      0.871631  0.843865  0.923543
h25       0.875487  0.832307  0.919221
L_mean_f  0.893750  0.847581  0.918863
h50f      0.894526  0.851424  0.918302
h50       0.897673  0.845485  0.918211
L_mean    0.899481  0.840543  0.917130
hMean     0.899481  0.840543  0.917130
qhMean    0.902685  0.840829  0.913642
h60       0.902154  0.844715  0.913641


Top 10 with negative correlation (Vol / ha)
               Mean DBH   BA / ha  Vol / ha
VDR           -0.729955 -0.806508 -0.836858
Skew          -0.715047 -0.863131 -0.800589
L_skewness    -0.699228 -0.850528 -0.774383
L_skewness_f  -0.677335 -0.806945 -0.721413
First_Return  -0.578153 -0.735066 -0.686424
Open_Prop     -0.542855 -0.770198 -0.652703
L_variation_f -0.443372 -0.562431 -0.649771
GapFrac       -0.494603 -0.720181 -0.623158
L_variation   -0.356848 -0.410068 -0.533406
LPI           -0.303634 -0.602969 -0.5257

In [9]:
# Sort by the DBH correlation column
df_corr.sort_values('BA / ha', ascending=False, inplace=True)

# Print the 10 highest values
print("Top 10 with positive correlation (BA / ha)")
print(df_corr.head(10))
print("\n")

# Sort by the DBH correlation column
df_corr.sort_values('BA / ha', ascending=True, inplace=True)

# Print the 10 lowest values
print("Top 10 with negative correlation (BA / ha)")
print(df_corr.head(10))
print("\n")

Top 10 with positive correlation (BA / ha)
                  Mean DBH   BA / ha  Vol / ha
chm_vf            0.840862  0.889441  0.944067
Oligophotic_Vol   0.828886  0.888714  0.913210
cvm_filled_vol    0.806828  0.877162  0.876274
Oligophotic_Prop  0.636937  0.873505  0.789017
h50f              0.894526  0.851424  0.918302
h60f              0.899793  0.848827  0.912498
L_mean_f          0.893750  0.847581  0.918863
h70f              0.902668  0.846543  0.907705
h75f              0.903823  0.845506  0.905233
h50               0.897673  0.845485  0.918211


Top 10 with negative correlation (BA / ha)
               Mean DBH   BA / ha  Vol / ha
Skew          -0.715047 -0.863131 -0.800589
L_skewness    -0.699228 -0.850528 -0.774383
L_skewness_f  -0.677335 -0.806945 -0.721413
VDR           -0.729955 -0.806508 -0.836858
Open_Prop     -0.542855 -0.770198 -0.652703
First_Return  -0.578153 -0.735066 -0.686424
GapFrac       -0.494603 -0.720181 -0.623158
LPI           -0.303634 -0.602969 -0.525768

In [10]:
# Sort by the DBH correlation column
df_corr.sort_values('Mean DBH', ascending=False, inplace=True)

# Print the 10 highest values
print("Top 10 with positive correlation (Mean DBH)")
print(df_corr.head(10))
print("\n")

# Sort by the DBH correlation column
df_corr.sort_values('Mean DBH', ascending=True, inplace=True)

# Print the 10 lowest values
print("Top 10 with negative correlation (Mean DBH)")
print(df_corr.head(10))
print("\n")

Top 10 with positive correlation (Mean DBH)
      Mean DBH   BA / ha  Vol / ha
h90   0.907691  0.833391  0.892995
h95   0.907625  0.827773  0.885551
h80   0.906667  0.840074  0.902401
h95f  0.906421  0.829111  0.884814
h75   0.906088  0.841299  0.905467
h90f  0.905427  0.834601  0.891714
h70   0.905383  0.842373  0.907951
h85f  0.905073  0.838910  0.896868
h80f  0.904752  0.842593  0.901564
h75f  0.903823  0.845506  0.905233


Top 10 with negative correlation (Mean DBH)
               Mean DBH   BA / ha  Vol / ha
VDR           -0.729955 -0.806508 -0.836858
Skew          -0.715047 -0.863131 -0.800589
L_skewness    -0.699228 -0.850528 -0.774383
L_skewness_f  -0.677335 -0.806945 -0.721413
First_Return  -0.578153 -0.735066 -0.686424
Open_Prop     -0.542855 -0.770198 -0.652703
GapFrac       -0.494603 -0.720181 -0.623158
wv_minmax     -0.487706 -0.518161 -0.498442
L_variation_f -0.443372 -0.562431 -0.649771
Euphotic_Prop -0.428465 -0.221944 -0.369181




From the ordered above, we can see that CHM volume (chm_vf) is the strongest linear predictor of stem volume (Pearson r = 0.95), however several other height metrics are also strongly correlated. If we choose to sort the Pearson correlation values by mean DBH, we get a different ordering of the most informative predictor variables.


![Volume v chm volume](figures/vol_v_chm_vf.png)

We can see from the scatter plot that the relationship between CHM volume and stem volume is approximately linear. However, there is evidence of heteroscedasticity; plots with higher CHM volumes exhibit greater variance in their stem volumes.

Heteroscedasticity is also evident in the relationship between the 95th discrete-return height percentile and mean DBH:

![DbH v 95th Height Percentile](figures/dbh_v_h95.png)


This reflects the fact that trees stop growing in height once they reach maturity (~ 20 m), however stem girth continues to increase with age.

From our initial data exploration, we have determined the following:
 1. There is a non-random (approximately linear) relationship between some of our LiDAR metrics and the forest inventory measurements.
 2. Some of the LiDAR metrics are weakly correlated with our response variables, therefore we shall have to omit these predictor variables from the regression model.
 3. There is evidence for heteroscedasticity in the data which may influence prediction accuracy.

*Heteroscedasticity is a hard word to pronounce, but it doesn't need to be a difficult concept to understand. Put simply, heteroscedasticity (also spelled heteroskedasticity) refers to the circumstance in which the variability of a variable is unequal across the range of values of a second variable that predicts it.*

# 6. Detecting Multicollinearity

One of the assumptions underpinning regression analysis is that a model should only contain predictor variables that are independent or exogeneous (i.e., the predictors should not be a function of other predictor variables already present within the model).

To measure multicollinearity between our predictor variables we have two options:
 1. Calculate the Pearson correlation coefficient (we could then choose to selectively remove predictors with r >= 0.9).
 2. Calculate the variance inflation factor (VIF) for each predictor (we could then selectively remove predictors with VIF scores >= 10).

The Pearson correlation coefficient provides a bivariate measure of collinearity (i.e., quantifying which two environmental variables are correlated with each another).

Conversely, VIF scores provide a more robust multivariate measure of collinearity (multicollinearity). Python code has been provided for you to calculate the VIF scores for each predictor variable:

In [11]:
vifs_series = rsgislib.tools.stats.calc_pandas_vif(df, cols=ind_vars)
vifs_series.to_csv('Forest_VIF_scores.csv')
vifs_series

Calculating VIF for 78 predictors variables...


N_Pulses         2.943701e+02
N_Returns        3.950012e+02
First_Return     0.000000e+00
Multi_Return     0.000000e+00
LPI              1.252028e+01
                     ...     
L_mean_f         1.562954e+06
L_scale_f        3.651456e+04
L_skewness_f     9.959568e+02
L_kurtosis_f     1.255413e+02
L_variation_f    8.271528e+02
Name: VIF, Length: 78, dtype: float64

## 6.1 Sort the VIF values

The code above exported the VIF scores into a comma-separated text file (Forest_VIF_scores.csv) that can be opened in your spreadsheet application or pandas as we did with the correlations. However, we can also sort them using pandas:

In [12]:
# Create dataframe from series
vifs_df = pandas.DataFrame({"VIF": vifs_series})

# Sort by the VIF column
vifs_df.sort_values('VIF', ascending=False, inplace=True)

# Print the 25 highest values
print("Top 25 VIF Scores (VIF)")
print(vifs_df.head(25))
print("\n")


# Sort by the VIF column
vifs_df.sort_values('VIF', ascending=True, inplace=True)

# Print the 25 lowest values
print("Lowest 25 VIF Scores (VIF)")
print(vifs_df.head(25))
print("\n")

Top 25 VIF Scores (VIF)
                      VIF
chm_vf       3.617349e+13
chm_vl       1.712395e+13
chm_vd       8.034968e+12
GapFrac      1.055731e+07
CDensity     1.051947e+07
L_mean_f     1.562954e+06
qhMean       8.058562e+05
Open_Prop    1.242767e+05
h70f         1.241849e+05
h75f         1.232850e+05
h85f         1.114476e+05
h60          1.104849e+05
h80          1.083900e+05
h70          1.058382e+05
h80f         9.515071e+04
h90          9.138985e+04
h50          8.916934e+04
h60f         8.027949e+04
h50f         7.021731e+04
h90f         6.349782e+04
h95          5.746946e+04
h99          3.965946e+04
Closed_Prop  3.699494e+04
L_scale_f    3.651456e+04
h95f         3.531742e+04


Lowest 25 VIF Scores (VIF)
                        VIF
wv_max             0.000000
Multi_Return       0.000000
First_Return       0.000000
hMean              0.000000
h25                0.000000
IQR                0.000000
h75                0.000000
cvm_filled_prop    0.000000
Oligophotic_Prop   

We can see from these results that many of the LiDAR metrics are highly correlated with each other since they have VIF scores well above 10. This indicates that we must select an uncorrelated subset of the predictor variables for the regression analysis. This leads on to the next notebook on feature selection.